In [43]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

In [44]:
today = date.today().strftime('%Y-%m-%d')
today

'2024-03-19'

In [45]:
# testing
today = '2024-03-18'

In [46]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate = '{today}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])

In [47]:
shortage_ext.loc[50,'Total Shortage Qty'] = 0

In [48]:
shortage_ext.loc[51,'Total Shortage Qty'] = 0

check last FD date is empty

In [49]:
lastFDnull = shortage_ext[shortage_ext['last FD date'].isnull()]
lastFDnull.loc[:,'Error'] = 'Last FD date is empty'

C:\Users\hsudu\AppData\Local\Temp\ipykernel_32228\1113314973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lastFDnull.loc[:,'Error'] = 'Last FD date is empty'


P1~P3 sum <> total shortage

In [50]:
QtyMismatch = []
for i, row in shortage_ext.iterrows():
    P123 = row['P1'] + row['Net P2'] + row['Net P3']
    total = row['Total Shortage Qty'] 
    if P123 != total:
        QtyMismatch.append(row)
    else:
        pass

if len(QtyMismatch) > 0:
    df_QtyMismatch = pd.DataFrame(QtyMismatch)
    df_QtyMismatch['Error'] = 'P1~P3 sum <> total shortage'


In [51]:
error_all = pd.concat([lastFDnull, df_QtyMismatch], axis=0)
error_all

,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,HP_PN,BuyerName,Error
185,CQQCI,1.0,HDD,2000GB 5400 SATA,Conti27I - 23C1,24.0,0.0,0.0,24.0,0.0,0.0,2024-03-18,NaT,"L01446-003,863128-004,L32157-002",TangWinnie,Last FD date is empty
50,CEI,CPU1347,CPU,AMD HAWKPOINT RYZEN5 8640HS FP8,Hyatt24C1/Park24C1,0.0,600.0,1000.0,0.0,0.0,0.0,2024-03-18,2024-03-23,N86918-A20,Ariel,P1~P3 sum <> total shortage
51,CEI,CPU1332,CPU,AMD PHOENIX RYZEN9 7940HS FP7R2,Thetiger23C1,0.0,0.0,300.0,0.0,0.0,0.0,2024-03-18,2024-04-13,N41421-A10,Celine,P1~P3 sum <> total shortage


In [52]:
To_list = 'gps.taiwan.nb.buy-sell@hp.com'
CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'
mail = win32.Dispatch("Outlook.Application").CreateItem(0)
mail.To = To_list
mail.CC = CC_list
mail.Subject = 'shortage dashboard issue as of ' + today
message = 'Hi team,<br><br>\
Please find the latest issues found in the shortage data.<br> Please use transformation tool to revise your data.<br><br>' + error_all.to_html(index=False)
mail.GetInspector 
index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 
mail.HTMLBody = mail.HTMLbody[:index + 1] + message + mail.HTMLbody[index + 1:] 
mail.display()